In [1]:
import os
import glob
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment', None)

In [2]:
# Combine all the CSV files
path = r'../data'
all_csv = glob.glob(path + "/*.csv")

temp = []
for csv in all_csv:
    df = pd.read_csv(csv, index_col=0, header=0, thousands=',')
    temp.append(df)

df = pd.concat(temp, axis=0, ignore_index=True)
df.head()

,Service,Type,Rating,Hires,Price,Zip Code
0,"MSF Landscapes, Masonry, and Irrigation",47,4.50,49.00,NaN,2903
1,Emi Ferreira Masonry,47,5.00,72.00,NaN,2903
2,27 Newman St,47,5.00,NaN,NaN,2903
3,JFLANDSCAPING,47,5.00,NaN,NaN,2903
4,Leaves land scape inc.,47,3.50,4.00,NaN,2903


In [3]:
df.shape

(69348, 6)

In [4]:
# Check and fix dtype
df.dtypes

Service      object
Type          int64
Rating      float64
Hires       float64
Price        object
Zip Code      int64
dtype: object

In [5]:
# See some prices values
df['Price'].unique()[:19]

# Need to fix that hour thing

array([nan, '75/hour', '60/hour', '495/hour', '28/hour', '50/hour',
       '65/hour', '45/hour', '125/hour', '35/hour', '40/hour', '70/hour',
       '39/hour', '55/hour', '85/service', '50/service', '150-$200',
       '15/hour', '36/hour'], dtype=object)

In [6]:
# See different types of payment
df['Price'] = df['Price'].astype(str)

tempList = []
checkUnique = []
for i in df["Price"]:
    x = i.split('/')
    tempList.append(x)
    
for i in tempList:
    try:
        x = i[1]
        checkUnique.append(x)
    except:
        pass

checkUnique = list(dict.fromkeys(checkUnique))
checkUnique

['hour',
 'service',
 'session',
 'on-site',
 'logo',
 'lesson',
 'walk',
 'cat',
 'person',
 'visit']

In [7]:
# Removed the hour into another column
tempPrice = []
tempPriceType = []
for i in df['Price']:
    if i.find("-") < 0: 
        x = i.split('/')
        tempPrice.append(x[0])
        try:
            y = x[1]
        except:
            y = ""
    else:
        x = i.split('/')
        x = x[0].split('-')
        tempPrice.append(x[0])
        try:
            y = x[1]
        except:
            y = ""
    tempPriceType.append(y)

# Add it to dataframe and check
df['Price'] = tempPrice
df['Price Type'] = tempPriceType
df['Price Type'] = df['Price Type'].astype(str)

# Convert price column to float
df['Price'] = df['Price'].str.replace(",","").astype(float)

In [8]:
# Double check data types
df.dtypes

Service        object
Type            int64
Rating        float64
Hires         float64
Price         float64
Zip Code        int64
Price Type     object
dtype: object

In [9]:
# Add the service type from database
serviceDf = pd.read_csv('../src/all_services.csv')

dfMerge = df.rename(columns={"Type": "ID"})
dfMerge = pd.merge(dfMerge, serviceDf, left_on='ID', right_on='ID',
                   how='left').drop('ID', axis=1)
df = dfMerge

In [10]:
zipCodes = pd.read_csv('../src/us_states.csv')

dfMerge = df
dfMerge = pd.merge(dfMerge, zipCodes, left_on='Zip Code', right_on='Representative ZIP Code',
                   how='left').drop('Representative ZIP Code', axis=1)

df = dfMerge

In [11]:
df.sample(5)

,Service,Rating,Hires,Price,Zip Code,Price Type,Services,Type,Segments,Capital City,State,Abbreviation
41579,First Choice Fit Personal Training,5.00,6.00,75.00,78701,,Personal Training,Main,Wellness,Austin,Texas,TX
56951,TutorWithTheBest,5.00,216.00,NaN,48226,,Statistical Data Analysis,Main,Business,Detroit,Michigan,MI
62360,Linda Stark Tutoring,5.00,8.00,45.00,68102,hour,Math Tutoring,Main,Lessons,Omaha,Nebraska,NE
14865,Nae's Sparkly Cleaning,NaN,NaN,NaN,30303,,House Cleaning,Main,Home Improvement,Atlanta,Georgia,GA
36613,Planting Seeds Tutoring & Test Prep- Houston,5.00,13.00,35.00,17101,hour,Reading and Writing Tutoring,Main,Lessons,"Harrisburg,",Pennsylvania,PA


In [12]:
# See shape
df.shape

(69348, 12)

In [13]:
# Check for duplicates
dfDuplicate = df[~df.duplicated("Service", keep='first')]
df = dfDuplicate

In [14]:
df.shape

(14656, 12)

In [15]:
# See overview of data
df.describe()

,Rating,Hires,Price,Zip Code
count,"13,708.00","10,399.00","8,920.00","14,656.00"
mean,4.89,45.71,120.88,"52,215.10"
std,0.31,95.78,186.70,"31,930.53"
min,1.00,2.00,1.00,"2,201.00"
25%,5.00,6.00,51.75,"23,219.00"
50%,5.00,16.00,80.00,"53,205.00"
75%,5.00,46.00,140.50,"83,702.00"
max,5.00,"2,041.00","7,000.00","99,801.00"


In [16]:
# Get implied revenue
df["Implied Revenue"] = df["Price"] * df["Hires"]

In [17]:
# See columns
print(df.columns.values)

['Service' 'Rating' 'Hires' 'Price' 'Zip Code' 'Price Type' 'Services'
 'Type' 'Segments' 'Capital City' 'State' 'Abbreviation' 'Implied Revenue']


In [18]:
# Rearrange columsn
df = df[[
    'Segments', 'Services', 'Type', 'Service', 'Hires', 'Price', 'Price Type',
    'Rating', 'Zip Code', 'Capital City', 'State', 'Abbreviation',
    'Implied Revenue'
]]

In [19]:
df.to_csv("../output/all_services.csv")